# Windowing Analysis: Visualizing Spline Transformations on Chest X-rays

This notebook demonstrates how the learned spline windowing function transforms chest X-ray images. We'll load sample images from the CheXpert dataset and show before/after comparisons of the windowing transformations.

## Overview
- Load sample chest X-ray images
- Apply learned windowing transformations
- Visualize before/after comparisons
- Analyze the effect on image contrast and features

In [ ]:
import os
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torchvision.transforms as transforms

# Add parent directory to path to import torchxrayvision
sys.path.insert(0, os.path.abspath('.'))
import torchxrayvision as xrv
from torchxrayvision.windowing import SplineWindowingFunction, create_windowing_model

print("Libraries imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

In [ ]:
# Load some sample chest X-ray images
# First, let's create a CheXpert dataset to get sample images
dataset_dir = "/lotterlab/datasets/"
print(f"Looking for datasets in: {dataset_dir}")

# Create CheXpert dataset
try:
    dataset = xrv.datasets.CheX_Dataset(
        imgpath=os.path.join(dataset_dir, "CheXpert-v1.0-small"),
        csvpath=os.path.join(dataset_dir, "CheXpert-v1.0-small/train.csv"),
        transform=None,  # We'll apply transforms manually
        views=["PA", "AP"]
    )
    print(f"Dataset loaded successfully! Total samples: {len(dataset)}")
    
    # Get a few sample images
    sample_indices = [0, 100, 500, 1000, 2000]  # Different types of cases
    sample_images = []
    sample_labels = []
    
    for idx in sample_indices:
        if idx < len(dataset):
            img, label = dataset[idx]
            sample_images.append(img)
            sample_labels.append(label)
    
    print(f"Loaded {len(sample_images)} sample images")
    print(f"Image shape: {sample_images[0].shape}")
    print(f"Image dtype: {sample_images[0].dtype}")
    print(f"Image range: [{sample_images[0].min():.3f}, {sample_images[0].max():.3f}]")
    
except Exception as e:
    print(f"Could not load CheXpert dataset: {e}")
    print("Will create synthetic test images instead...")
    
    # Create synthetic chest X-ray-like images for demonstration
    sample_images = []
    for i in range(5):
        # Create a synthetic image with chest-like features
        img = torch.randn(1, 512, 512) * 0.2 + 0.5
        # Add some structure (ribs, heart shadow, etc.)
        center_y, center_x = 256, 256
        y, x = torch.meshgrid(torch.arange(512), torch.arange(512), indexing='ij')
        
        # Heart shadow
        heart_mask = ((x - center_x + 50)**2 + (y - center_y + 80)**2) < 8000
        img[0][heart_mask] *= 0.7
        
        # Rib shadows
        for rib_y in range(150, 400, 40):
            rib_mask = torch.abs(y - rib_y) < 5
            img[0][rib_mask] *= 0.8
            
        sample_images.append(img)
    
    print(f"Created {len(sample_images)} synthetic sample images")
    print(f"Image shape: {sample_images[0].shape}")
    print(f"Image range: [{sample_images[0].min():.3f}, {sample_images[0].max():.3f}]")

In [ ]:
# Create different windowing functions to compare

# 1. Identity windowing (no transformation)
identity_windowing = SplineWindowingFunction(n_knots=10, learnable=True, init_nonlinear=False)

# 2. Non-linear spline windowing (initial state)
nonlinear_windowing = SplineWindowingFunction(n_knots=32, learnable=True, init_nonlinear=True)

# 3. Try to load a learned windowing function from a checkpoint if available
learned_windowing = None
checkpoint_path = "outputs/chex_full_windowing/chex-resnet50-chex_full_windowing-best_checkpoint.pt"

if os.path.exists(checkpoint_path):
    print(f"Loading learned windowing from: {checkpoint_path}")
    try:
        checkpoint = torch.load(checkpoint_path, map_location='cpu')
        # Extract windowing parameters if they exist
        if 'windowing_params' in checkpoint:
            print("Found windowing parameters in checkpoint!")
            windowing_params = checkpoint['windowing_params']
            print(f"Windowing type: {windowing_params.get('type', 'unknown')}")
        else:
            print("No windowing parameters found in checkpoint")
    except Exception as e:
        print(f"Could not load checkpoint: {e}")
else:
    print(f"No checkpoint found at {checkpoint_path}")

print("Windowing functions created!")

# Let's visualize the windowing functions
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Plot identity windowing
x_vals, y_vals = identity_windowing.visualize_mapping(n_points=200)
axes[0].plot(x_vals, y_vals, 'b-', linewidth=2, label='Identity Windowing')
axes[0].plot([0, 1], [0, 1], 'k--', alpha=0.5, label='Perfect Identity')
axes[0].set_title('Identity Windowing Function')
axes[0].set_xlabel('Input Intensity')
axes[0].set_ylabel('Output Intensity')
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].set_xlim(0, 1)
axes[0].set_ylim(0, 1)

# Plot non-linear windowing
x_vals, y_vals = nonlinear_windowing.visualize_mapping(n_points=200)
axes[1].plot(x_vals, y_vals, 'r-', linewidth=2, label='Non-linear Spline Windowing')
axes[1].plot([0, 1], [0, 1], 'k--', alpha=0.5, label='Identity')
axes[1].set_title('Non-linear Spline Windowing Function')
axes[1].set_xlabel('Input Intensity')
axes[1].set_ylabel('Output Intensity')
axes[1].legend()
axes[1].grid(True, alpha=0.3)
axes[1].set_xlim(0, 1)
axes[1].set_ylim(0, 1)

plt.tight_layout()
plt.show()

print("Windowing functions visualized!")

In [ ]:
# Apply windowing transformations to sample images

def apply_windowing_to_images(images, windowing_function, title_suffix=""):
    """Apply windowing function to a list of images and return results"""
    windowed_images = []
    
    with torch.no_grad():
        for img in images:
            # Ensure image is in correct range [0, 1]
            img_normalized = (img - img.min()) / (img.max() - img.min() + 1e-8)
            
            # Apply windowing
            windowed_img = windowing_function(img_normalized)
            windowed_images.append(windowed_img)
    
    return windowed_images

# Apply different windowing functions
print("Applying windowing transformations...")

identity_results = apply_windowing_to_images(sample_images, identity_windowing, "Identity")
nonlinear_results = apply_windowing_to_images(sample_images, nonlinear_windowing, "Non-linear Spline")

print(f"Applied windowing to {len(sample_images)} images")

# Create a comprehensive visualization
def show_windowing_comparison(original_images, identity_images, nonlinear_images, num_samples=3):
    """Show before/after comparison of windowing transformations"""
    
    num_samples = min(num_samples, len(original_images))
    fig, axes = plt.subplots(num_samples, 3, figsize=(15, 5*num_samples))
    
    if num_samples == 1:
        axes = axes.reshape(1, -1)
    
    for i in range(num_samples):
        # Original image
        img_orig = original_images[i][0].numpy() if isinstance(original_images[i], torch.Tensor) else original_images[i]
        axes[i, 0].imshow(img_orig, cmap='gray', vmin=0, vmax=1)
        axes[i, 0].set_title(f'Original Image {i+1}')
        axes[i, 0].axis('off')
        
        # Identity windowing
        img_id = identity_images[i][0].numpy() if isinstance(identity_images[i], torch.Tensor) else identity_images[i]
        axes[i, 1].imshow(img_id, cmap='gray', vmin=0, vmax=1)
        axes[i, 1].set_title(f'Identity Windowing {i+1}')
        axes[i, 1].axis('off')
        
        # Non-linear windowing
        img_nl = nonlinear_images[i][0].numpy() if isinstance(nonlinear_images[i], torch.Tensor) else nonlinear_images[i]
        axes[i, 2].imshow(img_nl, cmap='gray', vmin=0, vmax=1)
        axes[i, 2].set_title(f'Non-linear Windowing {i+1}')
        axes[i, 2].axis('off')
        
        # Add intensity statistics
        orig_stats = f"Range: [{img_orig.min():.3f}, {img_orig.max():.3f}]\nMean: {img_orig.mean():.3f}"
        id_stats = f"Range: [{img_id.min():.3f}, {img_id.max():.3f}]\nMean: {img_id.mean():.3f}"
        nl_stats = f"Range: [{img_nl.min():.3f}, {img_nl.max():.3f}]\nMean: {img_nl.mean():.3f}"
        
        axes[i, 0].text(0.02, 0.98, orig_stats, transform=axes[i, 0].transAxes, 
                       verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
        axes[i, 1].text(0.02, 0.98, id_stats, transform=axes[i, 1].transAxes, 
                       verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
        axes[i, 2].text(0.02, 0.98, nl_stats, transform=axes[i, 2].transAxes, 
                       verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    plt.tight_layout()
    plt.show()

# Show the comparison
print("Creating before/after comparison...")
show_windowing_comparison(sample_images, identity_results, nonlinear_results, num_samples=3)

print("Windowing comparison complete!")

In [ ]:
# Analyze intensity distributions and histograms

def analyze_intensity_distributions(original_images, windowed_images, windowing_name):
    """Analyze and plot intensity distributions before/after windowing"""
    
    fig, axes = plt.subplots(2, 2, figsize=(12, 8))
    fig.suptitle(f'Intensity Distribution Analysis - {windowing_name}', fontsize=16)
    
    # Flatten all images for histogram analysis
    orig_pixels = torch.cat([img.flatten() for img in original_images])
    wind_pixels = torch.cat([img.flatten() for img in windowed_images])
    
    # Plot histograms
    axes[0, 0].hist(orig_pixels.numpy(), bins=50, alpha=0.7, color='blue', label='Original')
    axes[0, 0].set_title('Original Image Histogram')
    axes[0, 0].set_xlabel('Pixel Intensity')
    axes[0, 0].set_ylabel('Frequency')
    axes[0, 0].grid(True, alpha=0.3)
    
    axes[0, 1].hist(wind_pixels.numpy(), bins=50, alpha=0.7, color='red', label='Windowed')
    axes[0, 1].set_title(f'{windowing_name} Histogram')
    axes[0, 1].set_xlabel('Pixel Intensity')
    axes[0, 1].set_ylabel('Frequency')
    axes[0, 1].grid(True, alpha=0.3)
    
    # Overlay histograms for comparison
    axes[1, 0].hist(orig_pixels.numpy(), bins=50, alpha=0.5, color='blue', label='Original', density=True)
    axes[1, 0].hist(wind_pixels.numpy(), bins=50, alpha=0.5, color='red', label=windowing_name, density=True)
    axes[1, 0].set_title('Histogram Overlay')
    axes[1, 0].set_xlabel('Pixel Intensity')
    axes[1, 0].set_ylabel('Density')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # Statistical comparison
    orig_stats = {
        'mean': orig_pixels.mean().item(),
        'std': orig_pixels.std().item(),
        'min': orig_pixels.min().item(),
        'max': orig_pixels.max().item(),
        'median': orig_pixels.median().item()
    }
    
    wind_stats = {
        'mean': wind_pixels.mean().item(),
        'std': wind_pixels.std().item(),
        'min': wind_pixels.min().item(),
        'max': wind_pixels.max().item(),
        'median': wind_pixels.median().item()
    }
    
    # Create statistics table
    stats_text = f"""
    Intensity Statistics:
    
    {'Metric':<10} {'Original':<12} {windowing_name:<15} {'Change':<10}
    {'─'*50}
    {'Mean':<10} {orig_stats['mean']:<12.4f} {wind_stats['mean']:<15.4f} {wind_stats['mean']-orig_stats['mean']:+.4f}
    {'Std':<10} {orig_stats['std']:<12.4f} {wind_stats['std']:<15.4f} {wind_stats['std']-orig_stats['std']:+.4f}
    {'Min':<10} {orig_stats['min']:<12.4f} {wind_stats['min']:<15.4f} {wind_stats['min']-orig_stats['min']:+.4f}
    {'Max':<10} {orig_stats['max']:<12.4f} {wind_stats['max']:<15.4f} {wind_stats['max']-orig_stats['max']:+.4f}
    {'Median':<10} {orig_stats['median']:<12.4f} {wind_stats['median']:<15.4f} {wind_stats['median']-orig_stats['median']:+.4f}
    """
    
    axes[1, 1].text(0.1, 0.9, stats_text, transform=axes[1, 1].transAxes, fontfamily='monospace',
                   verticalalignment='top', fontsize=9)
    axes[1, 1].set_title('Statistical Comparison')
    axes[1, 1].axis('off')
    
    plt.tight_layout()
    plt.show()

print("Analyzing intensity distributions...")

# Analyze non-linear windowing
analyze_intensity_distributions(sample_images, nonlinear_results, "Non-linear Spline")

print("Intensity distribution analysis complete!")

In [ ]:
# Interactive exploration of windowing parameters

def create_custom_windowing(n_knots=16, contrast_boost=1.0, brightness_shift=0.0):
    """Create a custom windowing function with adjustable parameters"""
    windowing_fn = SplineWindowingFunction(n_knots=n_knots, learnable=True, init_nonlinear=True)
    
    # Modify the spline weights to simulate different windowing effects
    with torch.no_grad():
        if hasattr(windowing_fn.spline, 'weights'):
            # Apply contrast and brightness adjustments to the weights
            weights = windowing_fn.spline.weights
            weights *= contrast_boost
            weights += brightness_shift
    
    return windowing_fn

def compare_windowing_effects(sample_img_idx=0):
    """Compare different windowing parameter effects on a single image"""
    
    if sample_img_idx >= len(sample_images):
        sample_img_idx = 0
    
    sample_img = sample_images[sample_img_idx]
    
    # Different windowing configurations
    windowing_configs = [
        {"n_knots": 8, "contrast_boost": 1.0, "brightness_shift": 0.0, "name": "Standard (8 knots)"},
        {"n_knots": 32, "contrast_boost": 1.0, "brightness_shift": 0.0, "name": "High Resolution (32 knots)"},
        {"n_knots": 16, "contrast_boost": 1.5, "brightness_shift": 0.0, "name": "High Contrast"},
        {"n_knots": 16, "contrast_boost": 0.7, "brightness_shift": 0.0, "name": "Low Contrast"},
        {"n_knots": 16, "contrast_boost": 1.0, "brightness_shift": 0.2, "name": "Bright Shift"},
        {"n_knots": 16, "contrast_boost": 1.0, "brightness_shift": -0.2, "name": "Dark Shift"}
    ]
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.flatten()
    
    # Normalize sample image
    img_normalized = (sample_img - sample_img.min()) / (sample_img.max() - sample_img.min() + 1e-8)
    
    for i, config in enumerate(windowing_configs):
        windowing_fn = create_custom_windowing(
            n_knots=config["n_knots"],
            contrast_boost=config["contrast_boost"],
            brightness_shift=config["brightness_shift"]
        )
        
        # Apply windowing
        with torch.no_grad():
            windowed_img = windowing_fn(img_normalized)
        
        # Display result
        img_display = windowed_img[0].numpy() if isinstance(windowed_img, torch.Tensor) else windowed_img
        axes[i].imshow(img_display, cmap='gray', vmin=0, vmax=1)
        axes[i].set_title(f'{config["name"]}\nRange: [{img_display.min():.3f}, {img_display.max():.3f}]')
        axes[i].axis('off')
        
        # Add small histogram
        from matplotlib.patches import Rectangle
        from matplotlib.patches import FancyBboxPatch
        
        # Create inset for histogram
        hist_data = img_display.flatten()
        counts, bins = np.histogram(hist_data, bins=20, range=(0, 1))
        
        # Add text with statistics
        stats_text = f"μ={img_display.mean():.3f}\nσ={img_display.std():.3f}"
        axes[i].text(0.02, 0.98, stats_text, transform=axes[i].transAxes,
                    verticalalignment='top', fontsize=8,
                    bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    plt.suptitle(f'Windowing Effects Comparison - Sample Image {sample_img_idx + 1}', fontsize=16)
    plt.tight_layout()
    plt.show()

print("Creating windowing effects comparison...")
compare_windowing_effects(sample_img_idx=0)

if len(sample_images) > 1:
    print("Comparing effects on a different image...")
    compare_windowing_effects(sample_img_idx=1)

In [ ]:
# Load and visualize learned windowing from training checkpoints

def load_learned_windowing_from_checkpoint(checkpoint_path):
    """Load learned windowing function from a training checkpoint"""
    
    if not os.path.exists(checkpoint_path):
        print(f"Checkpoint not found: {checkpoint_path}")
        return None
    
    try:
        print(f"Loading checkpoint: {checkpoint_path}")
        checkpoint = torch.load(checkpoint_path, map_location='cpu')
        
        # Check if this is a model with windowing
        model_state = checkpoint.get('model_state_dict', checkpoint)
        
        # Look for windowing function parameters
        windowing_keys = [k for k in model_state.keys() if 'windowing_function' in k]
        
        if windowing_keys:
            print(f"Found {len(windowing_keys)} windowing parameters")
            
            # Extract windowing parameters
            spline_weights = None
            knot_x = None
            
            for key in windowing_keys:
                if 'spline.weights' in key:
                    spline_weights = model_state[key]
                elif 'spline.knot_x' in key:
                    knot_x = model_state[key]
            
            if spline_weights is not None:
                print(f"Loaded spline weights: shape {spline_weights.shape}")
                
                # Create a new windowing function with the learned parameters
                n_knots = len(spline_weights) + 1 if spline_weights is not None else 10
                windowing_fn = SplineWindowingFunction(n_knots=n_knots, learnable=True, init_nonlinear=False)
                
                # Load the learned weights
                with torch.no_grad():
                    if hasattr(windowing_fn.spline, 'weights'):
                        windowing_fn.spline.weights.data = spline_weights.clone()
                    if knot_x is not None and hasattr(windowing_fn.spline, 'knot_x'):
                        windowing_fn.spline.knot_x.data = knot_x.clone()
                
                return windowing_fn
            else:
                print("No spline weights found in checkpoint")
                return None
        else:
            print("No windowing function parameters found in checkpoint")
            return None
            
    except Exception as e:
        print(f"Error loading checkpoint: {e}")
        return None

# Try to load learned windowing from different possible checkpoint locations
checkpoint_locations = [
    "outputs/chex_full_windowing/chex-resnet50-chex_full_windowing-best_checkpoint.pt",
    "outputs/chex_full_windowing/chex-resnet50-chex_full_windowing-e1_checkpoint.pt",
    "outputs/chex_windowing_viz/chex-resnet50-chex_windowing_viz-best_checkpoint.pt",
    "outputs/test_windowing/chex-resnet50-test_windowing-best_checkpoint.pt"
]

learned_windowing = None
for checkpoint_path in checkpoint_locations:
    if os.path.exists(checkpoint_path):
        print(f"\\nTrying checkpoint: {checkpoint_path}")
        learned_windowing = load_learned_windowing_from_checkpoint(checkpoint_path)
        if learned_windowing is not None:
            print("✅ Successfully loaded learned windowing function!")
            break

if learned_windowing is None:
    print("\\n❌ No learned windowing function found in any checkpoints.")
    print("This is expected if training hasn't completed yet.")
    print("The notebook will continue with synthetic windowing functions.")
else:
    print("\\n🎉 Learned windowing function loaded successfully!")
    
    # Visualize the learned windowing function
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Plot learned windowing curve
    x_vals, y_vals = learned_windowing.visualize_mapping(n_points=200)
    axes[0].plot(x_vals, y_vals, 'g-', linewidth=3, label='Learned Windowing Function')
    axes[0].plot([0, 1], [0, 1], 'k--', alpha=0.5, label='Identity')
    axes[0].set_title('Learned Windowing Function')
    axes[0].set_xlabel('Input Intensity')
    axes[0].set_ylabel('Output Intensity')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    axes[0].set_xlim(0, 1)
    axes[0].set_ylim(0, 1)
    
    # Compare learned vs. initial windowing
    x_vals_init, y_vals_init = nonlinear_windowing.visualize_mapping(n_points=200)
    axes[1].plot(x_vals_init, y_vals_init, 'r-', linewidth=2, label='Initial Windowing', alpha=0.7)
    axes[1].plot(x_vals, y_vals, 'g-', linewidth=3, label='Learned Windowing')
    axes[1].plot([0, 1], [0, 1], 'k--', alpha=0.5, label='Identity')
    axes[1].set_title('Learned vs Initial Windowing')
    axes[1].set_xlabel('Input Intensity')
    axes[1].set_ylabel('Output Intensity')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    axes[1].set_xlim(0, 1)
    axes[1].set_ylim(0, 1)
    
    plt.tight_layout()
    plt.show()
    
    # Apply learned windowing to sample images
    if len(sample_images) > 0:
        print("\\nApplying learned windowing to sample images...")
        learned_results = apply_windowing_to_images(sample_images, learned_windowing, "Learned")
        
        # Show comparison with learned windowing
        print("Creating comparison with learned windowing...")
        
        fig, axes = plt.subplots(min(3, len(sample_images)), 4, figsize=(20, 5*min(3, len(sample_images))))
        if len(sample_images) == 1:
            axes = axes.reshape(1, -1)
        
        for i in range(min(3, len(sample_images))):
            # Original
            img_orig = sample_images[i][0].numpy()
            axes[i, 0].imshow(img_orig, cmap='gray', vmin=0, vmax=1)
            axes[i, 0].set_title(f'Original {i+1}')
            axes[i, 0].axis('off')
            
            # Identity
            img_id = identity_results[i][0].numpy()
            axes[i, 1].imshow(img_id, cmap='gray', vmin=0, vmax=1)
            axes[i, 1].set_title(f'Identity {i+1}')
            axes[i, 1].axis('off')
            
            # Initial non-linear
            img_nl = nonlinear_results[i][0].numpy()
            axes[i, 2].imshow(img_nl, cmap='gray', vmin=0, vmax=1)
            axes[i, 2].set_title(f'Initial Spline {i+1}')
            axes[i, 2].axis('off')
            
            # Learned
            img_learned = learned_results[i][0].numpy()
            axes[i, 3].imshow(img_learned, cmap='gray', vmin=0, vmax=1)
            axes[i, 3].set_title(f'Learned Spline {i+1}')
            axes[i, 3].axis('off')
        
        plt.tight_layout()
        plt.show()
        
        print("Learned windowing analysis complete! 🎉")

## Summary and Conclusions

This notebook demonstrates the effect of learnable spline windowing functions on chest X-ray images. Here's what we observed:

### Key Findings:

1. **Windowing Function Shapes**: 
   - Identity windowing preserves the original intensity distribution
   - Non-linear spline windowing can enhance contrast and adjust brightness
   - Learned windowing functions adapt to the specific dataset characteristics

2. **Image Enhancement**:
   - Spline windowing can improve contrast in specific intensity ranges
   - Different spline parameters create different enhancement effects
   - The learned parameters depend on the dataset and training objectives

3. **Intensity Distribution Effects**:
   - Windowing can reshape the histogram of pixel intensities
   - This can help normalize differences between different X-ray machines/protocols
   - Useful for removing dataset-specific artifacts while preserving clinical information

### Next Steps:

1. **Training Analysis**: Run the full training to see how windowing parameters evolve
2. **Clinical Evaluation**: Assess whether windowing improves diagnostic accuracy
3. **Comparison Studies**: Compare different windowing initialization strategies
4. **Ablation Studies**: Measure the impact of windowing vs. no windowing on model performance

### Files Generated:
- Windowing visualization plots are saved during training to: `outputs/experiment_name/`
- Checkpoints with windowing parameters: `outputs/experiment_name/*checkpoint.pt`